In [35]:
import pandas as pd
import numpy as np

#read in employment files
years = list(range(2013, 2023))
employmentTables = []
for year in years:
    fileName = ["Employment Summary_", str(year), ".xlsx"]
    fileName = "".join(fileName)
    file = pd.read_excel(fileName)
    employmentTables.append(file)
    
#add year column to employment files
year = 2012
for i in range(0,10):
    employmentTables[i]["Year"] = year
    year += 1
    
#cleaning column names for consistency
allYears = pd.concat(employmentTables[0:10])
allYears.columns = allYears.columns.str.replace("FullTimeLongTerm", "FTLT")
allYears.columns = allYears.columns.str.replace("PartTimeLongTerm", "PTLT")
allYears.columns = allYears.columns.str.replace("FullTimeShortTerm", "FTST")
allYears.columns = allYears.columns.str.replace("PartTimeShortTerm", "PTST")

In [36]:
#pivoting from wide to long

#full time long term df
FTLTCols = [ colNm for colNm in (allYears.columns) if "PT" not in colNm and "ST" not in colNm]
FullTimeLT = allYears.loc[:, allYears.columns.isin(FTLTCols)].copy()
FullTimeLT.insert(loc=1, column='term', value="longterm")
FullTimeLT.insert(loc=1, column='time', value="fulltime")
FullTimeLT.columns = FullTimeLT.columns.str.replace("FTLT", "")

#full time short term df
FTSTCols = [ colNm for colNm in (allYears.columns) if "PT" not in colNm and "LT" not in colNm]
FullTimeST = allYears.loc[:, allYears.columns.isin(FTSTCols)].copy()
FullTimeST.insert(loc=1, column='term', value="shortterm")
FullTimeST.insert(loc=1, column='time', value="fulltime")
FullTimeST.columns = FullTimeST.columns.str.replace("FTST", "")

#part time long term df
PTLTCols = [ colNm for colNm in (allYears.columns) if "FT" not in colNm and "ST" not in colNm]
PartTimeLT = allYears.loc[:, allYears.columns.isin(PTLTCols)].copy()
PartTimeLT.insert(loc=1, column='term', value="longterm")
PartTimeLT.insert(loc=1, column='time', value="parttime")
PartTimeLT.columns = PartTimeLT.columns.str.replace("PTLT", "")

#part time short term df
PTSTCols = [ colNm for colNm in (allYears.columns) if "FT" not in colNm and "LT" not in colNm]
PartTimeST = allYears.loc[:, allYears.columns.isin(PTSTCols)].copy()
PartTimeST.insert(loc=1, column='term', value="shortterm")
PartTimeST.insert(loc=1, column='time', value="parttime")
PartTimeST.columns = PartTimeST.columns.str.replace("PTST", "")

#append dfs
allYears = pd.concat([PartTimeST, PartTimeLT, FullTimeLT, FullTimeST])

allYears.columns


Index(['SchoolName', 'time', 'term', 'Employed_BarPassageRequired',
       'Employed_BarPassageRequiredNumber', 'Employed_JDAdvantagePTLST',
       'Employed_JDAdvantageNumber', 'Employed_ProfessionPosition',
       'Employed_ProfessionPositionNumber', 'Employed_NonProfessionPosition',
       'Employed_NonProfessionPositionNumber', 'Employed_Undeterminable',
       'Employed_UndeterminableNumber', 'PursuingGraduateDegreeNumber',
       'UnEmployedStartDateDeferredNumber', 'UnEmployedNotSeekingNumber',
       'UnEmployedSeekingNumber', 'EmploymentStatusUnknownNumber',
       'Total_GraduatesNumber', 'Funded_BarPassge',
       'Funded_BarPassgeTotalEmployed', 'Funded_JDAdvantage',
       'Funded_JDAdvantageTotalEmployed', 'Funded_Profession',
       'Funded_ProfessionTotalEmployed', 'Funded_NONProfessional',
       'Funded_NONProfessionalTotalEmployed', 'Funded_Tot',
       'Funded_TotTotalEmployed', 'Solo-', 'Solo', '2-10-', '2-10', '11-25-',
       '11-25', '26-50-', '26-50', '51-100-'

In [37]:
#remove irrelevant columns

#remove summary statistic columns
summaryColumns = [colNm for colNm in (allYears.columns) if "Number"  in colNm]
summaryColumns
allYears = allYears.drop(summaryColumns, axis = 1)

#remove summary statistic columns
allYears = allYears.drop(['1-10', '101-250', '11-25', '2-10', '251-500', '26-50', '501-PLUS',  '51-100', 'Solo'], axis = 1)
allYears = allYears.drop(['PublicInterest', 'BusinessIndustry', 'Unknown', 'Government', 'PublicInterest',
                         'Clerkships_Federal', 'Clerkships_StateLocal', 'Clerkships_Other',
                        'EmployerTypeUnknown', 'Total', 'Clerkships_Tribal', 'Education',
                         'Clerkships_International'], axis = 1)

#remove state statistics
allYears = allYears.drop(['FirstLargestEmployment',
       'SecondLargestEmployment', 'ThirdLargestEmployment',
       'EmployedInForeignCountries'], axis = 1)

#remove university funded breakdown columns
allYears = allYears.drop(['Funded_BarPassge', 'Funded_BarPassgeTotalEmployed', 'Funded_JDAdvantage', 
                          'Funded_JDAdvantageTotalEmployed', 'Funded_Profession', 'Funded_ProfessionTotalEmployed',
                         'Funded_TotTotalEmployed', 'Funded_TotTotalEmployed', 
                          'Funded_NONProfessional', 'Funded_NONProfessionalTotalEmployed', 
                          'Funded_OtherPosition', 'Funded_OtherPosition',
                          'Funded_OtherPositionTotalEmployed', 'Funded_Tot'], axis =1)


In [38]:
#merge columns with name/category changes

#solo vs 2-10 law firms
allYears['1-10-'] = allYears['1-10-'].add(allYears['2-10-'], fill_value = 0).add(allYears['Solo-'], fill_value =0)
allYears = allYears.drop(['2-10-', 'Solo-'], axis =1)

#clerkships other broken out to intl and tribal in 2021, merge
allYears["Clerkships_Other_"] = allYears["Clerkships_Other_"].add(allYears["Clerkships_Tribal_"],fill_value =0).add(allYears["Clerkships_International_"], fill_value = 0)
allYears = allYears.drop(["Clerkships_Tribal_", "Clerkships_International_", "Employed_LawSchool"], axis =1)

#other nameing convention change
allYears["Employed_OtherPosition"] = allYears["Employed_NonProfessionPosition"].add(allYears["Employed_OtherPosition"], fill_value = 0)
allYears = allYears.drop(["Employed_NonProfessionPosition"], axis =1)
allYears.columns

Index(['SchoolName', 'time', 'term', 'Employed_BarPassageRequired',
       'Employed_JDAdvantagePTLST', 'Employed_ProfessionPosition',
       'Employed_Undeterminable', '11-25-', '26-50-', '51-100-', '101-250-',
       '251-500-', '501-', 'Unknown-', 'BusinessIndustry_', 'Government_',
       'PublicInterest_', 'Clerkships_Federal_', 'Clerkships_StateLocal_',
       'Clerkships_Other_', 'Education_', 'EmployerTypeUnknown_', 'Total_',
       'Year', '1-10-', 'Employed_OtherPosition', 'Employed_JDAdvantage'],
      dtype='object')

In [39]:
#rearrange columns
allYears = allYears.reindex(columns= ['SchoolName', 'Year','time', 'term', 'Employed_BarPassageRequired',
       'Employed_JDAdvantage', 'Employed_ProfessionPosition','Employed_OtherPosition', 'Employed_by_LawSchool',
       'Employed_Undeterminable', 'EnrolledInGraduateStudies','Unemployed', 'status_Unknown','1-10-', '11-25-', '26-50-', '51-100-', '101-250-',
       '251-500-', '501-', 'BusinessIndustry_', 'Government_',
       'PublicInterest_', 'Clerkships_Federal_', 'Clerkships_StateLocal_',
       'Clerkships_Other_', 'Education_', 'EmployerTypeUnknown_', 'Total_'])
allYears["status_Unknown"]

0     NaN
1     NaN
2     NaN
3     NaN
4     NaN
       ..
191   NaN
192   NaN
193   NaN
194   NaN
195   NaN
Name: status_Unknown, Length: 8056, dtype: float64

In [40]:
#split out just employer data
employer = allYears[['SchoolName', 'Year','time', 'term', '1-10-', '11-25-', '26-50-', '51-100-', '101-250-',
       '251-500-', '501-', 'BusinessIndustry_', 'Government_',
       'PublicInterest_', 'Clerkships_Federal_', 'Clerkships_StateLocal_',
       'Clerkships_Other_', 'Education_', 'EmployerTypeUnknown_']].copy()

employer


,SchoolName,Year,time,term,1-10-,11-25-,26-50-,51-100-,101-250-,251-500-,501-,BusinessIndustry_,Government_,PublicInterest_,Clerkships_Federal_,Clerkships_StateLocal_,Clerkships_Other_,Education_,EmployerTypeUnknown_
0,"AKRON, UNIVERSITY OF",2012,parttime,shortterm,0.0,0,0,0,0,0,0,0,0,2,0,0,0.0,1,0
1,"ALABAMA, UNIVERSITY OF",2012,parttime,shortterm,0.0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0
2,ALBANY LAW SCHOOL OF UNION UNIVERSITY,2012,parttime,shortterm,4.0,0,0,0,0,0,0,8,1,1,0,0,0.0,1,1
3,AMERICAN UNIVERSITY,2012,parttime,shortterm,4.0,1,1,0,0,0,0,18,2,6,0,0,0.0,6,3
4,APPALACHIAN SCHOOL OF LAW,2012,parttime,shortterm,1.0,0,0,0,0,0,0,2,0,0,0,0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191,WILLAMETTE UNIVERSITY,2021,fulltime,shortterm,0.0,0,0,0,0,0,1,0,0,0,0,2,0.0,0,0
192,WILLIAM AND MARY LAW SCHOOL,2021,fulltime,shortterm,0.0,0,1,1,0,0,0,0,1,0,0,0,0.0,0,0
193,"WISCONSIN, UNIVERSITY OF",2021,fulltime,shortterm,0.0,0,0,0,0,0,1,1,1,0,1,1,0.0,0,0
194,"WYOMING, UNIVERSITY OF",2021,fulltime,shortterm,0.0,0,0,0,0,0,0,0,0,0,1,0,0.0,0,0


In [41]:
employer = employer.rename(columns = {'1-10-': 'Law Firm 1-10', '11-25-': 'Law Firm 11-25', 
                                       '26-50-': 'Law Firm 26-50', '51-100-': 'Law Firm 51-100',
                                      '101-250-': 'Law Firm 101-250', '251-500-': 'Law Firm 251-500',
                                      '501-': 'Law Firm 500+'})
employer.columns = employer.columns.str.replace("_", "")
employer

,SchoolName,Year,time,term,Law Firm 1-10,Law Firm 11-25,Law Firm 26-50,Law Firm 51-100,Law Firm 101-250,Law Firm 251-500,Law Firm 500+,BusinessIndustry,Government,PublicInterest,ClerkshipsFederal,ClerkshipsStateLocal,ClerkshipsOther,Education,EmployerTypeUnknown
0,"AKRON, UNIVERSITY OF",2012,parttime,shortterm,0.0,0,0,0,0,0,0,0,0,2,0,0,0.0,1,0
1,"ALABAMA, UNIVERSITY OF",2012,parttime,shortterm,0.0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0
2,ALBANY LAW SCHOOL OF UNION UNIVERSITY,2012,parttime,shortterm,4.0,0,0,0,0,0,0,8,1,1,0,0,0.0,1,1
3,AMERICAN UNIVERSITY,2012,parttime,shortterm,4.0,1,1,0,0,0,0,18,2,6,0,0,0.0,6,3
4,APPALACHIAN SCHOOL OF LAW,2012,parttime,shortterm,1.0,0,0,0,0,0,0,2,0,0,0,0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191,WILLAMETTE UNIVERSITY,2021,fulltime,shortterm,0.0,0,0,0,0,0,1,0,0,0,0,2,0.0,0,0
192,WILLIAM AND MARY LAW SCHOOL,2021,fulltime,shortterm,0.0,0,1,1,0,0,0,0,1,0,0,0,0.0,0,0
193,"WISCONSIN, UNIVERSITY OF",2021,fulltime,shortterm,0.0,0,0,0,0,0,1,1,1,0,1,1,0.0,0,0
194,"WYOMING, UNIVERSITY OF",2021,fulltime,shortterm,0.0,0,0,0,0,0,0,0,0,0,1,0,0.0,0,0


In [42]:
#wide to long
employer = pd.melt(employer, 
                id_vars=['SchoolName', 'Year', 'time', 'term'], 
                value_vars= ['Law Firm 1-10', 'Law Firm 11-25',
       'Law Firm 26-50', 'Law Firm 51-100', 'Law Firm 101-250',
       'Law Firm 251-500', 'Law Firm 500+', 'BusinessIndustry', 'Government',
       'PublicInterest', 'ClerkshipsFederal', 'ClerkshipsStateLocal',
       'ClerkshipsOther', 'Education', 'EmployerTypeUnknown'])
employer = employer.rename(columns = {'variable': 'employer_type', 'value': 'number_grads'})
employer

,SchoolName,Year,time,term,employer_type,number_grads
0,"AKRON, UNIVERSITY OF",2012,parttime,shortterm,Law Firm 1-10,0.0
1,"ALABAMA, UNIVERSITY OF",2012,parttime,shortterm,Law Firm 1-10,0.0
2,ALBANY LAW SCHOOL OF UNION UNIVERSITY,2012,parttime,shortterm,Law Firm 1-10,4.0
3,AMERICAN UNIVERSITY,2012,parttime,shortterm,Law Firm 1-10,4.0
4,APPALACHIAN SCHOOL OF LAW,2012,parttime,shortterm,Law Firm 1-10,1.0
...,...,...,...,...,...,...
120835,WILLAMETTE UNIVERSITY,2021,fulltime,shortterm,EmployerTypeUnknown,0.0
120836,WILLIAM AND MARY LAW SCHOOL,2021,fulltime,shortterm,EmployerTypeUnknown,0.0
120837,"WISCONSIN, UNIVERSITY OF",2021,fulltime,shortterm,EmployerTypeUnknown,0.0
120838,"WYOMING, UNIVERSITY OF",2021,fulltime,shortterm,EmployerTypeUnknown,0.0


In [43]:
employer.to_csv("/Users/natdeacon/Desktop/GitHub/Law_School_Outcomes_Economics/employerClean.csv")